### Quick start : compare runs, choose a model and deply it to rest API

In this quick start will learn:
- Run a hyperparameter sweep on a training script 
- compare the resukts of the runs in the Ml flow UI
- choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform

![alt text](<Screenshot 2025-06-18 114110.png>)

In [1]:
import keras 
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,hp,tpe,fmin
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import mlflow
from mlflow.models import infer_signature


In [2]:
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
    )
data



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
## split the data into training, validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [4]:
train_x=train.drop(columns=["quality"],axis=1).values
train_y=train["quality"].values.ravel() ## ravel() is used to convert 1D array to 1D array

## validation dataset
test_x=test.drop(columns=["quality"],axis=1).values
test_y=test["quality"].values.ravel()

## split the training dataset into training and validation sets
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)


In [5]:
## ANN Model

def train_model(params, epochs,train_x,train_y,valid_x,valid_y, test_x,test_y):
    ## define the model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## train the model
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,epochs=epochs,batch_size=64,validation_data=(valid_x,valid_y))
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)
        eval_rmse=eval_result[1]

        ## log the parameters, metrics and model
        mlflow.log_params(params)
        mlflow.log_metric("rmse",eval_rmse)


        ## log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}


In [6]:
def objective(params):
    result=train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
        )
    return result

In [7]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}


In [8]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    ## conduct the hyperparameter search using hyperopt
    trails=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trails,
    )

    ## Fetch the details of the best run
    best_run=sorted(trails.results,key=lambda x:x["loss"])[0]

    ## log the best parameters, metrics and model
    mlflow.log_params(best)
    mlflow.log_metric("rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    ## print out the best parameters and metrics

    print(f"Best parameters: {best}")
    print(f"Best RMSE: {best_run['loss']}")

    

2025/06/18 13:29:29 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1:19 2s/step - loss: 31.4638 - root_mean_squared_error: 5.6093
15/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 12.2126 - root_mean_squared_error: 3.3715 
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.0032 - root_mean_squared_error: 2.8536 
30/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.2872 - root_mean_squared_error: 2.7272
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.7028 - root_mean_squared_error: 2.6207
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 6.7971 - root_mean_squared_error: 2.4485
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 6.3555 - root_mean_squared_error: 2.3611 - val_loss: 0.7567 - val_root_mean_squared_error: 0.8699

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.8799 - root_mean_squared_error: 0.9380
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7156 - root_mean_squared_error: 0.8447 
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

2025/06/18 13:29:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - loss: 40.1248 - root_mean_squared_error: 6.3344
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 30.8792 - root_mean_squared_error: 5.5386
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 26.9463 - root_mean_squared_error: 5.1538
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23.9594 - root_mean_squared_error: 4.8384
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 20.6638 - root_mean_squared_error: 4.4639
39/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 18.5079 - root_mean_squared_error: 4.2017
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17.8329 - root_mean_squared_error: 4.1165
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 16.8422 - root_mean_squared_error: 3.9891 - val_loss: 2.2676 - val_root_mean_squared_error: 1.5058

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 7s 177ms/step - loss: 2.1759 - root_mean

2025/06/18 13:30:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1:25 2s/step - loss: 33.8800 - root_mean_squared_error: 5.8207
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 32.8545 - root_mean_squared_error: 5.7317 
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 32.4509 - root_mean_squared_error: 5.6964
17/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 32.3075 - root_mean_squared_error: 5.6837
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 32.0707 - root_mean_squared_error: 5.6628
28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 31.7935 - root_mean_squared_error: 5.6381
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 31.4999 - root_mean_squared_error: 5.6119
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.2161 - root_mean_squared_error: 5.5863 
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 31.0062 - root_mean_squared_error: 5.5673 - val_loss: 25.4535 - val_root_mean_squared_error: 5.0451

Epoch 2/3                                          

2025/06/18 13:30:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [01:09<00:00, 17.49s/trial, best loss: 0.7567449808120728]

2025/06/18 13:30:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.02714698005517418), 'momentum': np.float64(0.6426812167781722)}
Best RMSE: 0.7567449808120728
